In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from ast import literal_eval

In [6]:
ratings =  pd.read_csv('ratings_small.csv')

**Collaborative Filtering**

In [7]:
#Preparing matrix for user-based and item-based
user_rating = ratings.pivot(index='userId', columns='movieId', values='rating')
# user_rating
avg_ratings = user_rating.mean(axis=1)
# avg_ratings
user_ratings_pivot = user_rating.sub(avg_ratings, axis=0)
# # user_based matrix
user_ratings_pivot.fillna(0, inplace=True)
# # Change from user_based to item_based matrix
movie_ratings_pivot = user_ratings_pivot.T
movie_ratings_pivot

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
movieId,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,1.648649,0.0,0.0
11,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.703276,0.0,0.00,0.0,0.0,0.000000,0.0,0.0
12,0.0,0.0,0.0,-0.348039,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0
13,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0
14,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426067,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0
426469,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0
433715,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0


In [8]:
#Matrix of similarity between each movie
similarities = cosine_similarity(movie_ratings_pivot)
cosine_similarity_df = pd.DataFrame(similarities,
                                    columns=movie_ratings_pivot.index,
                                    index=movie_ratings_pivot.index)
cosine_similarity_df

movieId,5,11,12,13,14,16,18,19,20,22,...,396152,402515,407887,408429,417859,426067,426469,433715,447027,459488
movieId,,,,,,,,,,,,,,,,,,,,,
5,1.000000,0.031113,0.000000,0.000000,0.065761,0.054881,0.000000,0.009426,0.005928,0.000000,...,0.054622,0.000000,-0.044256,0.000000,0.000000,0.000000,0.013457,-0.031842,-0.143325,0.011867
11,0.031113,1.000000,0.000000,0.000000,0.185475,0.025529,0.000000,-0.029667,0.000000,0.000000,...,0.134839,0.000000,-0.174237,0.000000,0.000000,0.000000,0.000000,0.000000,-0.029692,0.000000
12,0.000000,0.000000,1.000000,0.034441,0.000000,0.000000,0.083458,0.000000,0.071299,0.155246,...,0.000000,-0.104779,0.000000,0.054872,0.111854,0.083297,-0.009038,-0.083964,0.000000,-0.009621
13,0.000000,0.000000,0.034441,1.000000,0.000000,0.000000,0.171376,0.000000,0.000000,-0.018764,...,0.000000,0.000000,0.000000,0.000000,-0.113734,0.000000,0.000000,0.000000,0.000000,0.000000
14,0.065761,0.185475,0.000000,0.000000,1.000000,0.457086,0.000000,0.270017,0.000000,0.000000,...,0.384316,0.000000,-0.255338,0.000000,0.000000,0.000000,0.000000,0.000000,-0.085833,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426067,0.000000,0.000000,0.083297,0.000000,0.000000,0.000000,0.016580,0.000000,0.065449,0.016696,...,0.000000,-0.220196,0.000000,0.047502,0.095369,1.000000,-0.041212,-0.064202,0.000000,0.037388
426469,0.013457,0.000000,-0.009038,0.000000,0.000000,0.000000,0.001711,0.000000,-0.066395,-0.003432,...,0.000000,0.030314,0.000000,-0.020768,-0.007686,-0.041212,1.000000,-0.090716,-0.014436,-0.140157
433715,-0.031842,0.000000,-0.083964,0.000000,0.000000,0.000000,0.015899,0.000000,-0.303788,-0.031883,...,0.000000,0.114798,0.000000,-0.069181,-0.071410,-0.064202,-0.090716,1.000000,0.049333,0.010161


In [13]:
def get_similar_movies(movieId,user_rating):
  similar_score = cosine_similarity_df[movieId]*user_rating
  similar_score = similar_score.sort_values(ascending = False)
  value= similar_score[:10]
  return value

get_similar_movies(19995,5)
# #List the top 10 movie which is most similar to movie 1
# cosine_similarity_df.loc[1].sort_values(ascending=False).head(10)

movieId
19995     5.000000
286939    2.741303
44770     2.671413
9367      2.544438
119450    1.842382
1865      1.489529
250902    1.386478
14022     1.383627
24869     1.367875
220490    1.324018
Name: 19995, dtype: float64